In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import * #BollingerBands, ATR, KeltnerChannels, RSI, MACD
from plotting import CandlePlot 
from getCandle import Rates
import MetaTrader5 as mt5


In [2]:
pair = "EURUSD"
NB_H1_CANDLES = 1000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 

In [4]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

4e-05

In [5]:
# in range Feature
df_an = BollingerBandsFeature(df_an)
df_an = ATRFeature(df_an)
df_an = KeltnerChannelsFeature(df_an)
df_an = RSIFeature(df_an)
df_an = MACDFeature(df_an)

df_an

,time,open,high,low,close,tick_volume,spread,real_volume,BB_MA,BB_UP,...,Feat_gains,Feat_wins_rma,Feat_losses_rma,Feat_RSI_14,MACD,SIGNAL_MACD,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST
0,2023-01-25 02:00:00,1.08899,1.08905,1.08810,1.08872,4773,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-25 03:00:00,1.08872,1.08941,1.08841,1.08931,4115,0,0,NaN,NaN,...,0.00059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-25 04:00:00,1.08931,1.09016,1.08929,1.08965,2949,0,0,NaN,NaN,...,0.00034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-25 05:00:00,1.08965,1.08995,1.08925,1.08973,3288,0,0,NaN,NaN,...,0.00008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-25 06:00:00,1.08972,1.09029,1.08971,1.08988,2338,0,0,NaN,NaN,...,0.00015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2023-03-23 13:00:00,1.08771,1.08803,1.08698,1.08759,5479,0,0,1.087483,1.094017,...,-0.00012,0.000493,0.000355,0.581389,0.002430,0.002915,-0.000485,0.002430,0.002915,-0.000485
995,2023-03-23 14:00:00,1.08761,1.08939,1.08695,1.08733,10488,0,0,1.087898,1.093274,...,-0.00026,0.000458,0.000348,0.567994,0.002165,0.002765,-0.000601,0.002165,0.002765,-0.000601
996,2023-03-23 15:00:00,1.08733,1.08930,1.08730,1.08872,8368,0,0,1.088346,1.091974,...,0.00139,0.000524,0.000323,0.618588,0.002043,0.002621,-0.000578,0.002043,0.002621,-0.000578
997,2023-03-23 16:00:00,1.08873,1.08952,1.08806,1.08912,8167,0,0,1.088480,1.091973,...,0.00040,0.000516,0.000300,0.631947,0.001956,0.002488,-0.000532,0.001956,0.002488,-0.000532


In [ ]:
n_slow=26
n_fast=12 
n_signal=9
df = df_an.copy()

ema_long = df.close.ewm(min_periods=n_slow, span=n_slow).mean()
ema_short = df.close.ewm(min_periods=n_fast, span=n_fast).mean()

df['MACD'] = ema_short - ema_long
df['SIGNAL_MACD'] = df.MACD.ewm(min_periods=n_signal, span=n_signal).mean()
df['HIST'] = df.MACD - df.SIGNAL_MACD
df


In [ ]:
df.HIST.min()

In [ ]:
df.Feat_RSI_14.max()

In [ ]:
df_an = BollingerBandsFeature(df_an)
df_an = ATR(df_an)
df_an = RSI(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [ ]:
df_an.RSI_14

In [ ]:
cp = CandlePlot(df_an, candles=True)
cp.show_plot(line_traces=['BB_MA','BB_UP','BB_LW'])

In [ ]:
cp = CandlePlot(df_an, candles=True)
cp.show_plot(sec_traces=['ATR_14'])

In [ ]:
cp = CandlePlot(df_an, candles=True)
cp.show_plot(line_traces=['EMA','KeUp','KeLo'])

In [ ]:
cp = CandlePlot(df_an.iloc[-100:], candles=True)
cp.show_plot(sec_traces=['RSI_14'])

In [ ]:
cp = CandlePlot(df_an.iloc[-100:], candles=True)
cp.show_plot(sec_traces=[])

In [ ]:
cp = CandlePlot(df_an, candles=False)
cp.fig.add_trace(go.Bar(
    x=cp.df_plot.sTime,
    y=cp.df_plot.HIST,
    name="HIST"
))
cp.show_plot(line_traces=['SIGNAL_MACD', 'MACD'], sec_traces=[])

In [ ]:
df_an.columns